In [1]:
#import pybullet as pyb
import json
import time
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations
from collections import OrderedDict


In [2]:
# domain = 'torque'
domain = 'velocity'
# domain = 'pose'

# time_scale = '_policy'
time_scale = '_sim'

if domain == 'torque':
    time_scale = ''

# Set the paths to the coactivation file and the log file
v_logs = "/home/nash/DeepMimic/output/" + domain + time_scale + ".dat"

# coactivation_file = "/home/nash/Downloads/Palmetto/Avinash/snake/caterpillar/pca_learn_ExcCoact_general_3d_9-14B/pca_euler_snakeV0_caterpillar_general_babble_0.86696607_9d.txt"

coactivation_file = "/home/nash/Downloads/Palmetto/Avinash/cheetah/run/pca_general_C5d-1/pca_euler_cheetahV1_run_general_babble_0.3131973_5d.txt"

In [3]:
with open(coactivation_file) as f:
    # Load Coactivatiion matrix of size [k, dof]
    data = json.load(f)
    coactivation_matrix = data["Basis"]
    coactivation_idx = data["coactivations"][0] if type(data["coactivations"]) is list else \
                       list(range(1, int(data["coactivations"].strip('D')) + 1))
    
print("coactivation_idx:", coactivation_idx)
    
with open(v_logs) as tl:
    # Load velocity/torque/poses data of size [samples, dof]
    vels = np.loadtxt(tl)

num_coactivations = len(coactivation_matrix)

metrics_dict = OrderedDict()

coactivation_idx: [1, 2, 3, 8, 11]


In [4]:
def getC_Hat(coactivations):
    # Input: coactivation matrix of size [k, dof]
    # Returns: (C' * C)⁻¹ * C'
    coactivation_ = np.transpose(coactivations, [1, 0]) # [d, k]
    coactivation_t = coactivations  # [k, d]
    ct_c = np.matmul(coactivation_t, coactivation_)
    ct_c_inv = np.linalg.inv(ct_c)
    ct_c_inv_c = np.matmul(ct_c_inv, coactivation_t)
    return ct_c_inv_c

def getError(vector, C):
    # Input: Vector : Any Query vector (velocity/torque) of dimension (dof, ) 
    # Input: C: coactivation matrix of size [k, dof]
    
    # Output: Error of the best fit model
    # E = T - AC => E = T - (C'.C)⁻¹.C'.vector

    ct_c_inv_c = getC_Hat(C)
    A = np.matmul(ct_c_inv_c, vector)
    fit = np.matmul(A, C)
    error = vector - fit
    return np.linalg.norm(error)/np.linalg.norm(vector)

def select_coactivations(coactivation_matrix, indices=None):
    # Selects individual coactivations from coactivation_matrix according to index list
    # indices must be 0-indexed!
    
    if indices is None:
        return coactivation_matrix
    else:
        return np.take(coactivation_matrix, indices, axis=0)

In [5]:
# Check residuals for individual coactivations

for i, idx in zip(range(num_coactivations), coactivation_idx):
    residuals = []
    for t in vels:
        selected = select_coactivations(coactivation_matrix, [i])
        err = getError(t, selected)
        residuals.append(err)
    mean = np.mean(residuals)
    std = np.std(residuals)
    median = np.median(residuals)

    stat_dict = OrderedDict()
    stat_dict['mean'] = mean
    stat_dict['std'] = std
    stat_dict['median'] = median
    
    metrics_dict[idx] = stat_dict
    
    print("Coactivation: {}, Residual Mean: {:.6f}, STD: {:.6f}".format(idx, mean, std))

print("\nSorted Coactivations by Residual Mean")
print("--------------------------------------")
for k, v in sorted(metrics_dict.items(), key=lambda k_v: k_v[1]['mean']):
    print("Coactivation:", k, " Mean:", v['mean'], "  STD:", v['std'])

Coactivation: 1, Residual Mean: 0.915416, STD: 0.133116
Coactivation: 2, Residual Mean: 0.852375, STD: 0.135696
Coactivation: 3, Residual Mean: 0.973265, STD: 0.038589
Coactivation: 8, Residual Mean: 0.922107, STD: 0.061801
Coactivation: 11, Residual Mean: 0.953899, STD: 0.042552

Sorted Coactivations by Residual Mean
--------------------------------------
Coactivation: 2  Mean: 0.8523747518303912   STD: 0.13569609252147746
Coactivation: 1  Mean: 0.9154163551616632   STD: 0.13311635078024625
Coactivation: 8  Mean: 0.9221065067741542   STD: 0.061800758658326384
Coactivation: 11  Mean: 0.9538989876189208   STD: 0.042551708256588446
Coactivation: 3  Mean: 0.9732645897692375   STD: 0.038588810953400166


In [6]:
# Check residual for a multiple coactivation

list_to_check = [2, 5]
selected = select_coactivations(coactivation_matrix, np.array(list_to_check) - 1)
residuals = []
for t in vels:
    residuals.append(getError(t, selected))
print(np.mean(residuals))

0.8008715070330583


In [7]:
# Check residuals for coactivations [1...k]
for i in range(1, num_coactivations + 1):
    residuals = []
    for t in vels:
        selected = select_coactivations(coactivation_matrix, np.arange(0, i))
        residuals.append(getError(t, selected))
    print("Coactivations: {}, Mean: {:.3f}, STD: {:.2f}".format(coactivation_idx[:i], np.mean(residuals), np.std(residuals)))

Coactivations: [1], Mean: 0.915, STD: 0.13
Coactivations: [1, 2], Mean: 0.755, STD: 0.18
Coactivations: [1, 2, 3], Mean: 0.720, STD: 0.18
Coactivations: [1, 2, 3, 8], Mean: 0.602, STD: 0.20
Coactivations: [1, 2, 3, 8, 11], Mean: 0.528, STD: 0.19


In [8]:
# Check residuals for all combinations of 'n_combo' coactivations

n_combo = 2
combs = combinations(np.arange(0, num_coactivations), n_combo) 
comb_pairs = combinations(np.array(coactivation_idx), n_combo) 
mean_dict = {}
std_dict = {}
for comb, comb_pair in zip(combs, comb_pairs):
    residuals = []
    selected = select_coactivations(coactivation_matrix, comb)
    for t in vels:
        # Loop over all query vectors and store the error
        residuals.append(getError(t, selected))
        
    # Record the mean and std residuals of all the vectors
    mean_residual = np.round(np.mean(residuals), 5)
    std_residual = np.std(residuals)
#     comb_one_indexed = tuple([(cmb + 1) for cmb in comb])
    comb_one_indexed = comb_pair
    mean_dict[comb_one_indexed] = mean_residual
    std_dict[comb_one_indexed] = std_residual

In [9]:
# Sort by mean residual error

sorted_items = sorted(mean_dict.items(), key=lambda x: x[1])


# Display top 15 combinations

for combo, means in sorted_items[:15]:    
    print("Coactivations : {}, Mean: {:.3f}, STD: {:.3f}".format(combo, means, std_dict[combo]))

Coactivations : (1, 2), Mean: 0.755, STD: 0.175
Coactivations : (2, 8), Mean: 0.756, STD: 0.166
Coactivations : (2, 11), Mean: 0.801, STD: 0.124
Coactivations : (2, 3), Mean: 0.821, STD: 0.141
Coactivations : (1, 8), Mean: 0.829, STD: 0.152
Coactivations : (1, 11), Mean: 0.865, STD: 0.136
Coactivations : (8, 11), Mean: 0.871, STD: 0.081
Coactivations : (1, 3), Mean: 0.886, STD: 0.138
Coactivations : (3, 8), Mean: 0.894, STD: 0.067
Coactivations : (3, 11), Mean: 0.926, STD: 0.059
